Crime Zone Heatmaps with Python and Folium
==========================================

Initialize a Folium map

Additional tile providers: http://leaflet-extras.github.io/leaflet-providers/preview/

In [ ]:
pip install folium

     ------------------------------------ 102.3/102.3 kB 979.2 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [7]:
import folium
charlotte_lat_lon = [ 35.2271, -80.8431 ]
m = folium.Map(
    location=charlotte_lat_lon,
    zoom_start=11,
    tiles='https://{s}.tile.opentopomap.org/{z}/{x}/{y}.png',
    attr='Map data: &copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors, <a href="http://viewfinderpanoramas.org">SRTM</a> | Map style: &copy; <a href="https://opentopomap.org">OpenTopoMap</a> (<a href="https://creativecommons.org/licenses/by-sa/3.0/">CC-BY-SA</a>)'
)
m
   

Import data from file

In [9]:
import pandas as pd
df = pd.read_csv("CMPD_Homicide.csv")

Access dataframe values

In [12]:
df.head()

,X,Y,OBJECTID,ID,CRIME_REPORT_ID,AGE,GENDER,RACE_ETHNICITY,CLEARANCE_STATUS,IPV,...,CMPD_PATROL_DIVISION,NPA,CLEARANCE_DATE,CIRCUMSTANCES,COUNCIL_DISTRICT,COMMISSION_DISTRICT,CORRIDOR,CALENDAR_YEAR,CALENDAR_MONTH,DATE_REPORTED
0,-9.003138e+06,4.182906e+06,1,1,20230311-2238-01,23.0,Male,Black,Open,NaN,...,Westover,383,NaN,Other/Unknown,6,5,NaN,2023,3,2023/03/11 00:00:00+00
1,-8.989303e+06,4.197152e+06,2,2,20230224-0530-00,43.0,Female,Black,Open,Intimate Partner,...,North Tryon,249,NaN,Domestic,1,3,NaN,2023,2,2023/02/24 00:00:00+00
2,-9.005036e+06,4.198649e+06,3,3,20230219-1557-01,17.0,Male,Black,Open,NaN,...,Freedom,385,2023/02/23 00:00:00+00,Robbery,2,2,NaN,2023,2,2023/02/19 00:00:00+00
3,-8.999272e+06,4.195354e+06,4,4,20230217-1025-00,NaN,Male,Black,Open,NaN,...,Central,340,2023/02/17 00:00:00+00,Robbery,2,4,NaN,2023,2,2023/02/17 00:00:00+00
4,-9.001061e+06,4.193458e+06,5,5,20230213-2347-01,23.0,Female,Black,Exceptionally Cleared,Intimate Partner,...,Central,339,2023/02/23 00:00:00+00,Domestic,3,2,NaN,2023,2,2023/02/13 00:00:00+00


Rename or drop columns.

In [13]:
columns = {
    'DATE_REPORTED': 'date',
    'CIRCUMSTANCES': 'offense',
    'WEAPON': 'weapon',
    'LATITUDE_PUBLIC':'Lat',
    'LONGITUDE_PUBLIC':'Long',
}
df = df.rename(columns=columns)
df = df[ list(columns.values()) ]


Deal with data types

In [16]:
type(df.date[0])
df.date = pd.to_datetime(df.date)
df = df.sort_values(by="date")
print(df.date[0:10])

709   2015-01-01 00:00:00+00:00
708   2015-01-05 00:00:00+00:00
707   2015-01-06 00:00:00+00:00
706   2015-01-08 00:00:00+00:00
705   2015-01-08 00:00:00+00:00
704   2015-01-20 00:00:00+00:00
703   2015-01-28 00:00:00+00:00
702   2015-01-29 00:00:00+00:00
700   2015-02-24 00:00:00+00:00
701   2015-02-24 00:00:00+00:00
Name: date, dtype: datetime64[ns, UTC]


Deal with null values.

In [18]:
df.weapon.value_counts()

Handgun                      416
Firearm                      128
Knife                         41
Rifle                         24
Physical                      21
Unknown                       14
Blunt Trauma                  12
Vehicle                       10
Shotgun                        9
Cutting instrument             5
Handgun, Rifle                 4
Strangulation                  3
Personal                       3
Blunt Trauma, Handgun, Kn      2
Firearms                       2
Handgun, Shotgun               1
Arson, Strangulation           1
Arson, Physical                1
Arson                          1
Rifle, Shotgun                 1
Blunt trauma, Firearm          1
Razor                          1
Blunt trauma                   1
Cutting instrument, Stran      1
Screwdriver                    1
Name: weapon, dtype: int64

Group data by month

In [22]:
import datetime
from dateutil.relativedelta import relativedelta

print(df.date.min())
print(df.date.max())

2015-01-01 00:00:00+00:00
2023-03-11 00:00:00+00:00


Create heatmap

In [28]:
from folium.plugins import HeatMapWithTime

m = folium.Map(charlotte_lat_lon, zoom_starts=11)
hm = HeatMapWithTime(
    data=[ m.values.tolist() for m in months ],
    radius=5,
    max_opacity=0.5,
    auto_play=False,
)
hm.add_to(m)
m